In [11]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.qingque import Qingque
from characters.silver_wolf import Silver_Wolf
from characters.fu_xuan import Fu_Xuan
from simulations.qingque_mono_quantum import QQ_Mono_Quantum, Initialize_QQ_Mono_Quantum
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table

# Character Imports

In [12]:
qingque_speed = 98

In [13]:
silver_wolf_speed = 161

In [14]:
sparkle_speed = 160.1

In [15]:
fu_xuan_speed = 134
fu_xuan_technique = True

# Simulations

In [16]:
arena, qq, spk, sw, fx = Initialize_QQ_Mono_Quantum(qingque_speed=qingque_speed, fu_xuan_speed=fu_xuan_speed, sparkle_speed=sparkle_speed, silver_wolf_speed=silver_wolf_speed)

In [17]:
action_dataframe = Action_Table()
probability_dataframe = Qingque_Probability_Table()

In [18]:
QQ_Mono_Quantum(arena, qq, spk, sw, fx, action_df=action_dataframe, prob_df= probability_dataframe)

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)
c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


# Results

## Action List

In [19]:
pd.set_option("display.max_rows", None)
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Silver Wolf,0.0,62.111801,6,55.00,Skill,-1
1,Sparkle,0.0,62.460962,5,55.00,Skill,-1
2,Qingque,0.0,62.560962,5,70.00,Enhanced Basic,1
3,Fu Xuan,0.0,74.626866,4,67.50,Basic,1
4,Silver Wolf,0.0,124.223602,5,55.00,Basic,1
5,Sparkle,0.0,124.921924,6,90.82,Skill,-1
6,Sparkle,0.0,124.921924,5,126.64,Ultimate,4
7,Qingque,0.0,125.021924,7,70.00,Enhanced Basic,1
8,Fu Xuan,0.0,149.253731,6,91.38,Basic,1
9,Silver Wolf,1.0,186.335404,7,55.00,Basic,1


## QQ Probability List

In [20]:
probability_dataframe_styled = probability_dataframe.df.style.set_caption(f"Qingque speed = {qingque_speed}, Sparkle speed = {sparkle_speed}, Silver Wolf Speed = {silver_wolf_speed}, Fu Xuan speed = {fu_xuan_speed}")
probability_dataframe_styled

,Cycle,Action Gauge,Current SP,Current Tiles,P(Success),Average # Skills,Most Likely # Skills,SP Consumed
0,0.000000,62.560962,5,3,0.967908,2.124346,2.284459,2.000000
1,0.000000,125.021924,7,4,0.995798,1.898525,2.239724,2.000000
2,1.000000,187.482886,6,4,0.990547,1.861762,2.226879,2.000000
3,1.000000,249.943848,2,4,0.762222,1.028889,1.666667,2.000000
4,2.000000,312.404809,6,4,0.990547,1.861762,2.226879,2.000000
5,3.000000,374.865771,6,4,0.990547,1.861762,2.226879,2.000000
6,3.000000,437.326733,3,3,0.838962,1.568573,2.053498,2.000000
7,4.000000,499.787695,7,4,0.995798,1.898525,2.239724,2.000000
8,5.000000,562.248657,5,4,0.978734,1.790886,2.197988,2.000000
9,5.000000,624.709619,3,4,0.892812,1.420658,1.987654,2.000000
